# Synthesize a coloring book

In [2]:
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage

from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches, Mm
import re

## Define keys and load models

In [4]:
# openAI key
openai.api_key = ""
# for azure openAI
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

## Inputs

In [6]:
title = "The Zoo Animals Playing Sports Coloring Book"
topic = "anthropormophic zoo animals playing sports"
number_pages = 50
images_to_generate = 5
# upscale image
enhance_images = False

## Helper functions

In [7]:
# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" ideas for a coloring book on the topic of "+topic+"/n/nIdeas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = remove_after_create(response.choices[0].text.strip())
    return result

# Create fun facts about the image
def generate_summary(prompt: str):
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.5,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result

# parse a string to list based on return characters
def parse_string(input_string: str) -> list:
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if not 'Create ' in line and not line == "":
            result.append(remove_number_period(line.strip()))
    return result

def remove_after_create(s: str) -> str:
    keyword = "Create "
    index = s.find(keyword)
    
    if index == -1:
        # If "Create " is not found, return the original string
        return s
    
    # Find the end index of the keyword
    end_index = index + len(keyword)
    
    # Return the string up to the end of the keyword
    return s[:end_index]

def remove_number_period(s: str) -> str:
    # Use a regular expression to match a number followed by a period and a space
    pattern = r'^\d+\.\s'
    
    # Remove the matched pattern and return the modified string
    return re.sub(pattern, '', s).strip()

def generate(plain_text: str) -> str:
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

## Workflow

In [8]:
# Create page ideas given the desired topic
page_ideas = generate_coloring_book_page_descriptions(topic, number_pages)
print(page_ideas)

('Anthropomorphic Zoo Animals Playing Sports\n\n1. Giraffe playing golf\n2. Elephant playing football\n3. Kangaroo playing basketball\n4. Polar bear playing hockey\n5. Lion playing baseball\n6. Chimpanzee playing soccer\n7. Gorilla playing cricket\n8. Hippopotamus playing rugby\n9. Rhinoceros playing lacrosse\n10. Panda playing badminton\n11. Bear playing volleyball\n12. Camel playing dodgeball\n13. Cheetah playing Frisbee\n14. Fox playing disc golf\n15. Wolf playing kickball\n16. Wombat playing tennis\n17. Ostrich playing squash\n18. Alligator playing racquetball\n19. Zebra playing softball\n20. Meerkat playing golf\n21. Monkey playing ping pong\n22. Armadillo playing ultimate frisbee\n23. Puma playing dodgeball\n24. Emu playing handball\n25. Koala playing floorball\n26. Wild boar playing tug-of-war\n27. Beaver playing curling\n28. Badger playing netball\n29. Rabbit playing table tennis\n30. Crow playing bocce ball\n31. Parrot playing volleyball\n32. Hawk playing skittles\n33. Osprey 

In [9]:
# parse the page ideas into a list for each page
pages = parse_string(page_ideas[0])
print(pages)

['Anthropomorphic Zoo Animals Playing Sports', 'Giraffe playing golf', 'Elephant playing football', 'Kangaroo playing basketball', 'Polar bear playing hockey', 'Lion playing baseball', 'Chimpanzee playing soccer', 'Gorilla playing cricket', 'Hippopotamus playing rugby', 'Rhinoceros playing lacrosse', 'Panda playing badminton', 'Bear playing volleyball', 'Camel playing dodgeball', 'Cheetah playing Frisbee', 'Fox playing disc golf', 'Wolf playing kickball', 'Wombat playing tennis', 'Ostrich playing squash', 'Alligator playing racquetball', 'Zebra playing softball', 'Meerkat playing golf', 'Monkey playing ping pong', 'Armadillo playing ultimate frisbee', 'Puma playing dodgeball', 'Emu playing handball', 'Koala playing floorball', 'Wild boar playing tug-of-war', 'Beaver playing curling', 'Badger playing netball', 'Rabbit playing table tennis', 'Crow playing bocce ball', 'Parrot playing volleyball', 'Hawk playing skittles', 'Osprey playing beach volleyball', 'Pelican playing croquet', 'Wart

In [10]:
# create a summary
prompt = "Create a summary for a coloring book about "+topic
summary = generate_summary(prompt)
summary

'This fun-filled coloring book features anthropomorphic zoo animals playing various sports! Kids will love bringing these adorable animals to life with vibrant colors. From a giraffe playing basketball to a rhinoceros playing soccer, these animals are sure to bring smiles to young and old alike. The book includes 20 pages of illustrations, each with a unique and entertaining sports scene. Let the games begin!'

In [11]:
# create fun facts and images for each page
fun_facts = []
images = []
count = 0
# iterate for each desired page
for page in pages:
    prompt = "photo, anthropormophic "+page
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt).images[0]
        # enhance/upscale the generated image
        if enhance_images:
            enhanced_image = enhance_image(image)
            # create a coloring book image based on the generated image
            coloring_book_image = generate_coloring_page(enhanced_image)
        else:
            coloring_book_image = generate_coloring_page(image)
        # save the image with the format image_pageNumber_imageNumber as jpg
        coloring_book_image.save("image_"+str(count)+"_"+str(image_num)+".jpg")
        image.save("photo_"+str(count)+"_"+str(image_num)+".jpg")
    # add the fun facts for each page in a list
    fun_facts.append(remove_after_create(generate_fun_facts(page)))
    count = count + 1

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
# write to a word document
# create a new docx document object
document = Document()

# set the document size and margins
section = document.sections[0]
section.page_width = Inches(6.0)
section.page_height = Inches(9.0)
section.left_margin = Inches(0.25)
section.right_margin = Inches(0.25)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)

# Add a title page
document.add_heading(title, 0)
p = document.add_paragraph('A coloring book about '+topic)
p = document.add_paragraph(summary)

count = 0
for page in pages:
    document.add_page_break()
    document.add_heading(pages[count], level=1)
    document.add_picture("image_"+str(count)+"_0.jpg", width=Inches(5.5))
    document.add_page_break()
    document.add_paragraph('Fun facts about '+pages[count], style='Intense Quote')
    document.add_paragraph(fun_facts[count])
    count += 1

document.save('demo.docx')